In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
df = yf.download("AAPL", start="2010-01-01")

[*********************100%***********************]  1 of 1 completed


In [3]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.505281,493729600
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.516526,601904800
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.412873,552160000
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.401018,477131200
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.443573,447610800


In [4]:
df["price"] = df.Open.shift(-1)

In [5]:
def ma_calc(n):
    df["sma"] = df.Close.rolling(n).mean()

In [7]:
ma_calc(10)
df

,Open,High,Low,Close,Adj Close,Volume,price,sma
Date,,,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.505281,493729600,7.664286,NaN
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.516526,601904800,7.656429,NaN
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.412873,552160000,7.562500,NaN
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.401018,477131200,7.510714,NaN
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.443573,447610800,7.600000,NaN
...,...,...,...,...,...,...,...,...
2023-03-06,153.789993,156.300003,153.460007,153.830002,153.830002,87558000,153.699997,148.491000
2023-03-07,153.699997,154.029999,151.130005,151.600006,151.600006,56182000,152.809998,148.803001
2023-03-08,152.809998,153.470001,151.830002,152.869995,152.869995,47204800,153.559998,149.199001


In [11]:
def back_test(df, n):
    ma_calc(n)

    in_position = False

    profits = []

    for index,row in df.iterrows():
        if not in_position:
            if row.Close > row.sma:
                buyprice = row.price
                in_position = True
        if in_position:
            if row.Close < row.sma:
                profit = (row.price - buyprice) / buyprice
                profits.append(profit)
                in_position = False

    gain = (pd.Series(profits) + 1).prod()

    return gain

In [12]:
back_test(df,10)

8.26166985752013

In [15]:
for n in np.arange(5,205,5):
    print(f"parameter: {n}")
    print(back_test(df,n))

parameter: 5
3.427864184572647
parameter: 10
8.26166985752013
parameter: 15
8.217817504068412
parameter: 20
14.740347777859364
parameter: 25
12.481029267585223
parameter: 30
9.180671321897552
parameter: 35
7.757010034630497
parameter: 40
7.973858273486889
parameter: 45
4.9391290080854215
parameter: 50
7.509142100258755
parameter: 55
6.475326740966538
parameter: 60
6.470466041403682
parameter: 65
6.714675774681963
parameter: 70
5.044996746542134
parameter: 75
4.778689138099906
parameter: 80
4.154083023460525
parameter: 85
4.877996420417991
parameter: 90
4.885941125527226
parameter: 95
4.958006458896084
parameter: 100
5.383057100766451
parameter: 105
6.147145669883576
parameter: 110
6.88379862485197
parameter: 115
7.408339727154173
parameter: 120
7.5867348217958135
parameter: 125
7.9200663108102
parameter: 130
8.579106878053913
parameter: 135
8.431212072468384
parameter: 140
9.668396779562073
parameter: 145
10.897552129340816
parameter: 150
13.062413594815556
parameter: 155
14.3513602865

### No trying with 2 MA instead of 1

In [16]:
def ma_calc(n,m):
    df["sma1"] = df.Close.rolling(n).mean()
    df["sma2"] = df.Close.rolling(m).mean()

In [17]:
def back_test(df, n, m):
    ma_calc(n, m)

    in_position = False

    profits = []

    for index,row in df.iterrows():
        if not in_position:
            if row.sma1 > row.sma2:
                buyprice = row.price
                in_position = True
        if in_position:
            if row.sma1 < row.sma2:
                profit = (row.price - buyprice) / buyprice
                profits.append(profit)
                in_position = False

    gain = (pd.Series(profits) + 1).prod()

    return gain

In [20]:
x = pd.DataFrame(np.arange(10,205,5))
y = pd.DataFrame(np.arange(10,205,5))

final = pd.merge(x,y, how = "cross")
final.columns = ["sma1", "sma2"]
final = final[final.sma1 != final.sma2]

In [ ]:
for n, m in final.values:
    print(f"parameter: {n}, {m}")
    print(back_test(df,n))